In [ ]:
# You will need to run this block twice to make it effective
!apt-get update 
!apt-get install cmake 
!pip install --upgrade setuptools 
!pip install ez_setup 
!pip install gym==0.24.1
!pip install gym[all]

!pip install gym pyvirtualdisplay 
!apt-get install -y xvfb python-opengl ffmpeg 

In [ ]:
from importlib import reload
import utils
reload(utils)
from utils import*

import gym
from gym.wrappers.monitoring import video_recorder
from gym.wrappers.monitoring.video_recorder import VideoRecorder
from gym.wrappers.record_video import RecordVideo
import glob
import io
import base64
from IPython.display import HTML
from pyvirtualdisplay import Display
from IPython import display as ipythondisplay

import torch
import torch.nn as nn
import numpy as np
from torch import nn
import copy
from collections import deque
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
import math

from torch import randint
from time import sleep
import pickle
import statistics as st
from gym.core import RewardWrapper
import gc



display = Display(visible=0, size=(1400, 900))
display.start()

"""
Utility functions to enable video recording of gym environment 
and displaying it.
To enable video, just do "env = wrap_env(env)""
"""


def simulate(agent=None,env=None,epsilon=0,memory=None,render=False):
  agent.eval()
  env.reset()
  if(render):
    env.env = RecordVideo(env.env, './video')
    env.env.render()
  state,rew,done,info = env.skip_episodes(70,[0,0.1,0])
  ep_len = 0
  while not done:
      # exploitation(0) vs exploration(1)
      sample = torch.bernoulli(torch.tensor(epsilon).float())
      if(sample==1):
        A = torch.randint(0,3,(1,))
      else:
        A = agent.get_action(state)

      # progress a time step
      next_state, rew, done, info = env.step(agent.convert_action(A,state))
      #plot_image(state)
      #print(rew)

      if(done):
        break

      # collect memory
      if(memory!=None):
        memory.collect([state, A, rew, next_state])
      state = next_state

      ep_len = env.ep_len
      # stop criteria 
      if(ep_len>2000):
        break

  # readd 100 to episode reward to resync measured reward with documentation (undo the -100 penalty)
  score = env.real_rew
  if(render):
    print("score",score,"ep_len",ep_len)
    env.env.close()
    show_video()
  
  return score,ep_len


def test_model(agent, env, episodes=1):
  rewards = []
  ep_lens = []
  for i in range(0,episodes):
    rew,ep_len = simulate(agent,env)
    rewards.append(rew)
    ep_lens.append(ep_len)
    print("Test "+str(i+1)+"/"+str(episodes)+": reward =",rew," episode len =",ep_len)
  print("\nAverage Reward = ",sum(rewards)/len(rewards),"Average Ep_len = ",sum(ep_lens)/len(ep_lens),"\n")
  return rewards,ep_lens


class ExperienceReplay(object):
# one entry is [state,action,reward,next_state]
  def  __init__(self, length):
    self.experience_replay = deque(maxlen=length)
  def collect(self,experience):
    self.experience_replay.append(experience)
    return
  def sample_from_experience(self, sample_size):
    sample_size = min(sample_size,len(self.experience_replay))
    sample = random.sample(self.experience_replay,sample_size)
    state = torch.tensor([episode[0] for episode in sample]).float()
    action = torch.tensor([episode[1] for episode in sample]).float()
    reward = torch.tensor([episode[2] for episode in sample]).float()
    next_state = torch.tensor([episode[3] for episode in sample]).float()

    return state,action,reward,next_state



class DQN_Network(nn.Module):
  def __init__(self,gamma = None):
    super().__init__()
    #layers
    self.LeakyReLU = nn.LeakyReLU()
    self.conv1 = nn.Conv2d(1,8,kernel_size = 7, stride = 4,padding = 0)
    self.conv2 = nn.Conv2d(8,16,kernel_size = 3, stride = 1,padding = 2)
    self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
    self.fc1 = nn.Linear(577,256)
    self.fc2 = nn.Linear(256,50)
    self.fc3 = nn.Linear(50,3)
    self.batchnormCNN1 = nn.BatchNorm2d(num_features = 8)
    self.batchnormCNN2 = nn.BatchNorm2d(num_features = 16)
    self.batchnormFC1 = nn.BatchNorm1d(num_features = 256)
    self.flatten = nn.Flatten()
    self.gamma = gamma
  def forward(self,x):
    # reformat image (input = BS,96,96, or 96,96) (output = BS,1,96,96)
    x = torch.from_numpy(np.ascontiguousarray(x)).float()
    if(x.dim()==2):
      x = torch.unsqueeze(x,dim=0)
      x = torch.unsqueeze(x,dim=0)
    elif(x.dim()==3):
      x = torch.unsqueeze(x,dim=1)
    subimage = (x[:,:,84:96,13:14]-0.495)*10
    speed = torch.sum(subimage,dim=(2,3))
    x = x[:,:,:84,:]
    #plot_image(np.squeeze(x.detach().numpy()))
    
    #print(x.shape)
    x = self.batchnormCNN1(self.LeakyReLU(self.conv1(x)))
    #print(x.shape)
    x = self.pool(x)
    #print(x.shape)
    x = self.batchnormCNN2(self.LeakyReLU(self.conv2(x)))
    #print(x.shape)
    x = self.pool(x)
    #print(x.shape)
    x = self.flatten(x)
    #print(x.shape)
    x = torch.cat((x,speed),dim=1)
    x = self.batchnormFC1(self.LeakyReLU(self.fc1(x)))
    #x = self.LeakyReLU(self.fc1(x))
    #print(x.shape)
    x = self.LeakyReLU(self.fc2(x))
    #print(x.shape)
    x = self.fc3(x) 
    #print(x.shape)
    return x
  def get_action(self,state):
    qvals = self.forward(state)
    return torch.argmax(qvals,1) 
  def convert_action(self,action,state):
    # determine if you are going too fast
    speed = get_speed(state).item()
    if(speed>3.5):
      accel = 0
    elif(speed>2.5):
      accel = 0
    else:
      accel = 0.1
    # convert action from index, to a list of turning,engine,breaking strengths
    action = action.item()
    # Discretized action space (left-forward,straight-forward,right-forward)
    if(action == 0):
      return [-0.3,accel,0]
    elif(action == 1):
      return [0,accel,0]
    elif(action == 2):
      return [0.3,accel,0]


### Training

In [ ]:
def load_memory(new,epsilon,exp_replay_size,initial_size=None):
  if(initial_size==None):
    initial_size=exp_replay_size
  # Create the model
  env = wrap_env(gym.make("CarRacing-v1").unwrapped)
  agent = DQN_Network()
  if(not new):
    agent.load_state_dict(torch.load("car-racing-dqn.pth"))
  memory = ExperienceReplay(exp_replay_size)

  # initiliaze experience replay
  index = 0
  for i in range(exp_replay_size):
      state = env.reset()
      simulate(agent,env,epsilon = epsilon, memory = memory)
      if(len(memory.experience_replay)>=initial_size):
        break
      print(len(memory.experience_replay))

  return memory


def update(agent,optimizer,loss_func,target_agent,memory,batch_size):
  agent.train()
  target_agent.eval()
  # current (S,A) Qval
  state,action,reward,next_state = memory.sample_from_experience(batch_size)
  Qvals = agent(state)
  curr_Qval = Qvals[torch.arange(Qvals.size(0)),action.long()]
  
  # best next (S,A) Qval
  with torch.no_grad():
    next_Qval, indices = torch.max(target_agent(next_state),dim=1)

  # update agent
  #print(reward + agent.gamma*next_Qval,curr_Qval)
  loss = loss_func(reward + agent.gamma*next_Qval, curr_Qval)
  loss.backward(retain_graph = False)
  optimizer.step()
  optimizer.zero_grad()


def train(new,num_ep,lr_start,epsilon_start,gamma,memory):
  # set hyperparamters
  agent = DQN_Network(gamma=gamma)
 
  # start new run
  if(new):
    reward_hist = []; ep_len_hist = []; lr_hist = []; epsilon_hist = []
  # load previous runs
  else:
    agent.load_state_dict(torch.load("car-racing-dqn.pth")); reward_hist = load_list("reward_hist.data");ep_len_hist = load_list("ep_len_hist.data");epsilon_hist = load_list("epsilon_hist.data");lr_hist = load_list("lr_hist.data")

  #initialize models
  target_agent = DQN_Network(agent.gamma)
  target_agent.load_state_dict(agent.state_dict())
  env = wrap_env(gym.make("CarRacing-v1").unwrapped)
  optimizer = torch.optim.SGD(agent.parameters(),lr_start)
  MSELoss = torch.nn.MSELoss()

  # training loop
  for ep_num in tqdm(range(0,num_ep)):
    lr = lr_start*(0.99042**ep_num)
    epsilon = epsilon_start*(0.99424**ep_num)

    for param_group in optimizer.param_groups:
      param_group['lr'] = lr

    state, done, losses, ep_len, reward = env.reset(), False, 0, 0, 0
    reward,ep_len = simulate(agent,env,epsilon = epsilon, memory = memory)       
  
    for i in range(0,30):
      update(agent,optimizer,MSELoss,target_agent,memory,batch_size=32)
    target_agent.load_state_dict(agent.state_dict())
    gc.collect(generation=2)

    if(ep_num%3==0):
      reward, ep_len = test_model(agent=agent,env=env, episodes=1)
      print("Settings: lr =",lr,"epsilon =",epsilon)
      print("Test Result: reward =",reward[0],"episode length =",ep_len[0])
      reward_hist.append(reward[0])
      ep_len_hist.append(ep_len[0])
      lr_hist.append(lr)
      epsilon_hist.append(epsilon)
    
    if(ep_num%30==0):
      # save results
      torch.save(agent.state_dict(),"car-racing-dqn.pth");save_list(reward_hist,"reward_hist.data");save_list(ep_len_hist,"ep_len_hist.data");save_list(epsilon_hist,"epsilon_hist.data");save_list(lr_hist,"lr_hist.data")

In [ ]:
memory = load_memory(new=True,epsilon=1,exp_replay_size=2000)
train(new=True,num_ep = 360,lr_start=0.0003,epsilon_start=0.8,gamma=0.92,memory=memory)

/usr/local/lib/python3.7/dist-packages/gym/utils/passive_env_checker.py:98: UserWarning: WARN: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  "We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) "


12
20
27
35
44
55
64
74
90
99
108
119
136
157
169
185
205
219
227
237
254
268
297
305
321
333
352
360
367
383
394
405
423
444
473
487
499
511
520
535
547
555
563
571
587
595
613
630
638
653
664
686
699
706
719
728
738
748
768
786
795
806
821
847
869
885
905
919
928
945
957
968
980
988
1010
1018
1029
1042
1051
1064
1083
1091
1100
1137
1164
1183
1195
1209
1225
1237
1254
1265
1276
1288
1299
1312
1329
1338
1346
1364
1377
1386
1394
1402
1411
1420
1432
1445
1457
1466
1479
1490
1501
1510
1520
1539
1562
1570
1580
1588
1597
1608
1625
1640
1667
1677
1690
1710
1733
1746
1758
1768
1777
1794
1802
1810
1824
1832
1846
1863
1878
1897
1913
1933
1945
1962
1979
1990


  0%|          | 1/360 [00:05<35:45,  5.98s/it]

Test 1/1: reward = 34.98156996587023  episode len = 125

Average Reward =  34.98156996587023 Average Ep_len =  125.0 

Settings: lr = 0.0003 epsilon = 0.8
Test Result: reward = 34.98156996587023 episode length = 125


  1%|          | 4/360 [00:20<31:12,  5.26s/it]

...

 99%|█████████▊| 355/360 [44:46<00:33,  6.71s/it]

Test 1/1: reward = 112.48411552346606  episode len = 245

Average Reward =  112.48411552346606 Average Ep_len =  245.0 

Settings: lr = 9.93549355278238e-06 epsilon = 0.10351087135637403
Test Result: reward = 112.48411552346606 episode length = 245


 99%|█████████▉| 358/360 [45:01<00:12,  6.09s/it]

Test 1/1: reward = 142.33170731707372  episode len = 280

Average Reward =  142.33170731707372 Average Ep_len =  280.0 

Settings: lr = 9.65267426410301e-06 epsilon = 0.10173248644495686
Test Result: reward = 142.33170731707372 episode length = 280


100%|██████████| 360/360 [45:11<00:00,  7.53s/it]


In [ ]:
env = wrap_env(gym.make("CarRacing-v1").unwrapped)
agent = DQN_Network()
agent.load_state_dict(torch.load("car-racing-dqn.pth"))

simulate(agent=agent,env=env,render=True)
#test_model(agent,env,episodes=10)

In [ ]:
print("lr_hist")
hist = load_list("lr_hist.data")
for stage in hist:
  print(stage)
print("\nepsilon_hist")
hist = load_list("epsilon_hist.data")
for stage in hist:
  print(stage)
print("\nep_len_hist")
hist = load_list("ep_len_hist.data")
for stage in hist:
  print(st.mean(stage))
print("\nreward_hist")
hist = load_list("reward_hist.data")
for stage in hist:
  print(st.mean(stage))

lr_hist
0.0005
0.0005
0.00025
0.00025
0.000125
0.000125
6.2e-05
6.2e-05
3.1e-05
3.1e-05
1.6e-05
1.6e-05

epsilon_hist


FileNotFoundError: ignored